In [ ]:
import random
from combination_generator import *
from IPython.display import HTML

class Player:
  def __init__(self, name, strategy = None):
    self.cards = []
    self.name = name
    self.strategy = strategy

  def give_card(self, card):
    self.cards.append(card)
  
  def play_hand(self, pile, used_cards, player_card_count):
    valid_combination = []
    combinations = combination_from_pile(self.cards)
    for c in combinations:
      if compare_combinations(pile, c) < 0:
        if pile == [] or len(pile[1]) == len(c[1]):
          valid_combination.append(c)
    if valid_combination == []:
      return None
    print(f"possible moves {len(valid_combination)}")

    # if pile is not empty then allow pass as a "move"
    if not pile == []:
      valid_combination.append(None)

    return self.strategy(valid_combination, pile, used_cards, player_card_count)

  def first_play(self, player_card_count, lowest_card = 0):
    valid_combinations = []
    combinations = combination_from_pile(self.cards)
    for c in combinations:
      _, cards = c
      if lowest_card in cards:
        valid_combinations.append(c)

    print(f"possible moves {len(valid_combinations)}")
    
    return self.strategy(valid_combinations, [], [], player_card_count)

In [ ]:
def dummy_strategy(valid_combinations, pile, user_cards, player_cards):
  return valid_combinations[0]

In [ ]:
def simulate_game(number_of_players = 4):
  deck = []
  pile = []
  for x in range(52):
    deck.append(x)

  random.shuffle(deck)
  print(deck)
  
  players = [Player(f"{p + 1}", dummy_strategy) for p in range(number_of_players)]

  def describe(cards):
    card_images = []
    for index in cards:
      suite_id = index // 13
      if suite_id == 0:
        suite_id = 2
      elif suite_id == 1:
        suite_id = 0
      elif suite_id == 2:
        suite_id = 1
      card_images.append(f"faces/{suite_id}_{((index + 2) % 13) + 1}.svg")
    imagesList=''.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black; background: white;' src='%s' />" % str(s) 
                for s in sorted(card_images) ])
    display(HTML(imagesList))


  def win_condition(player_list):
    for p in player_list:
      if len(p.cards) == 0:
        return True
    return False

  for x in range(13):
    for p in players:
      p.give_card(deck.pop())
  
  # print hands
  lowest_card = 52
  start_player = players[0]
  print(f"starting hands:")
  for p_num in range(len(players)):
    p = players[p_num]
    print(f"For Player {p.name}:")
    describe(p.cards)
    print(f"")
    p.cards.sort()
    if p.cards[0] < lowest_card:
      lowest_card = p.cards[0]
      start_player = p

  players.remove(start_player)
  turn_order = [start_player]
  for p in players:
    turn_order.append(p)

  current_turn = 0
  last_play = None
  last_move = None

  print(f"Game start!")
  print("-------------")
  used_cards = []
  while not win_condition(turn_order):
    current_player = turn_order[current_turn % number_of_players]
    print(" ")
    print(f"turn {current_turn} =============================")
    print(f"player {current_player.name} cards in hand {len(current_player.cards)}:")
    if last_move == None and last_play == current_player:
      pile = []
    
    player_card_counts = [-1 if current_player == p else len(p.cards) for p in turn_order]

    if current_turn == 0:
      move = current_player.first_play(player_card_counts, lowest_card)
    else:
      move = current_player.play_hand(pile, used_cards, player_card_counts)
      
    if move != None:
      pile = move
      last_play = current_player
      last_move = move
      _, cards = move
      describe(cards)
      for c in cards:
        current_player.cards.remove(c)
        used_cards.append(c)
      if len(current_player.cards) == 0:
        print(f"Player {current_player.name} has won")
    else:
      print('pass')
      last_move = None
    current_turn += 1
  for p in turn_order:
    if len(p.cards) > 0:
      print(f"Player #{p.name} Cards left: {len(p.cards)}")
    

Start a game simulation

In [ ]:

simulate_game(4)


In [ ]:

# pile = []
# for x in range(52):
#   pile.append(x)
  
# result = combination_from_pile(pile)


# total_combinations = len(result)

# ratios = []
# print(f"total combinations = {total_combinations}")
# for type in range(9):
#   count = len([x for x in filter(lambda x: x[0] == type, result)])
#   print(f"{type}: count = {count} / {total_combinations}")
#   ratio = count / total_combinations
#   ratios.append([type, ratio])
# print(ratios)

# for type, cards in result:
#   print(f"comb type {type}")
#   for c in cards:
#     _, suite, card, index = card_id_to_name(c)
#     print(f"{suite} {card}")